In [1]:
from osgeo import gdal


In [2]:
path_img = 'E:\QGIS Data\s2_chenggong_6bands.tif'
path_imgr = "E:\QGIS Data\sample\s2_chenggong_6bands_resam.tif"

In [3]:
dset = gdal.Open(path_img)


#### 重采样到60m分辨率

In [4]:
geo_trans = dset.GetGeoTransform()
geo_trans
x_min = geo_trans[0]
y_max = geo_trans[3]
x_max = x_min + dset.RasterXSize*geo_trans[1]
y_min = y_max - dset.RasterYSize*geo_trans[5]
print(x_min,x_max,y_min,y_max)

874420.0 904220.0 2802200.0 2769600.0


In [5]:
x_res_resam = 60
y_res_resam = -60
x_size = (x_max-x_min)/x_res_resam
y_size = (y_max-y_min)/y_res_resam
print(x_size,y_size)

496.6666666666667 543.3333333333334


In [6]:
x_size_update = round(x_size)
y_size_update = round(y_size)
print(x_size_update,y_size_update)


497 543


In [7]:
x_res_update = (x_max - x_min)/x_size_update
y_res_update = -(y_max - y_min)/y_size_update
print(x_res_update,y_res_update)


59.95975855130785 60.03683241252302


In [8]:
geotrans_resam = [x_min, x_res_resam, 0, y_max, 0, y_res_resam]




In [19]:
driver = gdal.GetDriverByName('GTiff')
dset_resam = driver.Create(path_imgr,xsize=x_size_update,ysize=y_size_update,bands=dset.RasterCount,eType=gdal.GDT_Int16)
dset_resam.SetProjection(dset.GetProjection())
dset_resam.SetGeoTransform(geotrans_resam)

#重采样
resam = gdal.ReprojectImage(src_ds=dset,dst_ds=dset_resam,eResampleAlg=gdal.GRA_Bilinear)
dset_resam = None




In [4]:
path_reproj_2 = "E:\QGIS Data\sample\projection2.tif"
path_resam_2 = "E:\QGIS Data\sample\c_resample.tif"


In [14]:
reproj = gdal.Translate(srcDS=path_img, destName=path_reproj_2, format = 'GTiff')

In [14]:
from osgeo import osr
wgs84_epsg = 4326
wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(wgs84_epsg)
wkt84 = wgs84.ExportToWkt()
print(wkt84)



In [10]:
import os
os.environ['PROJ_LIB']='E:\python\Lib\site-packages\pyproj\proj_dir\share\proj'

In [17]:
reproj = gdal.Warp(destNameOrDestDS=path_reproj_2, srcDSOrSrcDSTab=path_img, dstSRS = 'EPSG:4326')

TypeError: in method 'wrapper_GDALWarpDestName', argument 4 of type 'GDALWarpAppOptions *'

In [30]:
resam = gdal.Warp(destNameOrDestDS=path_resam_2, srcDSOrSrcDSTab=path_img, xRes=60,yRes=60,resampleAlg=gdal.GRA_Bilinear)
